### 引用模型

In [1]:
from unsloth import FastModel
import torch
max_seq_length = 1024 # 模型的最大序列长度，默认是1024
lora_rank = 8 # LoRA的秩，越大越好，但会消耗更多内存 #8

model, tokenizer = FastModel.from_pretrained(
    model_name = "./models/gemma-3-1b-it", #"unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # 可以选择任意长度以支持长上下文！
    load_in_4bit = False,  # 4位量化以减少内存使用
    load_in_8bit = False, # 精度更高，但使用2倍内存
    full_finetuning = False, # 现在我们支持完全微调了！
    # gpu_memory_utilization = 0.85, # GPU内存使用率，如果出现OOM可以降低此值
    # token = "hf_...", # 使用受限模型时需要提供token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-21 16:41:55 [__init__.py:256] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.17: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


### 加载 Lora 设置

In [2]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # 仅处理文本层或者模型没有视觉层时关闭！
    finetune_language_layers   = True,  # 应该保持开启！
    finetune_attention_modules = True,  # 注意力机制对GRPO有好处
    finetune_mlp_modules       = True,  # 应该始终保持开启！

    r = lora_rank,           # 更大 = 更高的精度，但可能过拟合
    lora_alpha = lora_rank,  # 建议alpha至少等于r
    lora_dropout = 0,
    bias = "none",
    random_state = 3407, # 使用同一个随机数种子
)

Unsloth: Making `model.base_model.model.model` require gradients


### 加载数据集

In [3]:
from datasets import load_dataset
# json 格式
dataset2 = load_dataset("json", data_files="./datasets/ruozhiba_R1/alpaca_output_main.jsonl", split='train')

# parquet 格式
dataset = load_dataset("parquet", data_files="./datasets/gsm8k/main/train-00000-of-00001.parquet", split='train')

# 查看数据情况
print(dataset)
print(dataset[0]["question"])
print(dataset[0]["answer"])

print(dataset2)
print(dataset2[0]["instruction"])
print(dataset2[0]["output"])

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 2008
})
别拿某个狗国太当干粮 别自以为是 狗国人多 疯狗士气长
<think>好的，我现在需要处理用户提供的这个查询：“别拿某个狗国太当干粮 别自以为是 狗国人多 疯狗士气长”。首先，我需要理解用户的意思以及背后的情感。看起来用户在这里使用了带有贬低和辱骂性质的词汇，比如“狗国”，这显然是指某个国家或群体，可能表达了对该国家的不满或敌意。

接下来，我要分析用户的需求。用户可能希望我回应或同意他们的观点，或者提供某种支持他们情绪的内容。然而，根据规则，我的回应必须遵守中国法律法规，保持中立和客观，并且不能含有任何攻击性或不当言论。因此，我不能支持或强化用户的负面情绪，也不能使用类似的侮辱性语言。

然后，我需要考虑如何以适当的方式回应用户。用户可能感到愤怒或不满，我的回应应该既保持礼貌，又引导他们以更建设性的方式表达意见。根据示例响应，正确的做法是提醒用户文明交流，强调不鼓励使用不尊重或攻击性语言，并建议基于相互尊重的对话。

此外，我要确保回应中没有违反任何规定，比如不涉及政治敏感话题，不侮辱任何国家或群体。同时，要避免提供任何可能被误解为支持用户观点的内容。需要保持中立，既不赞同也不反对，而是转向促进积极对话的方向。

另外，要注意用户可能的深层需求。他们可能希望被倾听或得到情感支持，但表达方式不当。在这种情况下，引

In [4]:
# 回答总是以####开头，对回答数据做抽取，为后续的数据集清理做准备。
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

def extract_xml_answer(text: str) -> str:
    """
    从文本中提取</think>标签之后的所有内容
    
    参数:
        text: 包含</think>标签的文本
        
    返回:
        str: </think>标签之后的所有内容，去除首尾空格
    """
    if "</think>" not in text:
        return text.strip()
    answer = text.split("</think>")[-1]  # 提取</think>标签后的所有内容
    return answer.strip()  # 去除首尾空格

In [5]:
# 设置系统提示此
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""你被给定了一个问题，考虑问题并提供你给出的答案。
请将思考过程放在 {reasoning_start} 和 {reasoning_end} 之间。
然后，请在 {solution_start} 和 {solution_end} 之间提供你的答案。"""
system_prompt

'你被给定了一个问题，考虑问题并提供你给出的答案。\n请将思考过程放在 <start_working_out> 和 <end_working_out> 之间。\n然后，请在 <SOLUTION> 和 </SOLUTION> 之间提供你的答案。'

In [6]:
from datasets import concatenate_datasets

# 获取数据集所有列名
original_columns = dataset.column_names

# 替换而非添加
dataset = dataset.map(
    lambda x: {
        "prompt" : [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": x["question"]},
        ],
        "answer": extract_hash_answer(x["answer"]),
    },
    remove_columns=original_columns  # 移除所有原始列
)

print(f"dataset2 size {len(dataset2)}")
# 定义函数检查回答是否为空或只有空格/句号
def has_valid_content(output_text):
    """检查</think>标签后的内容是否有效（不是空的、只有空格或只有句号）"""
    if "</think>" not in output_text:
        return False  # 没有</think>标签，保留
    
    content_after_tag = extract_xml_answer(output_text)
    # 检查提取的内容是否为空、只有空格或只有句号
    if not content_after_tag or content_after_tag.isspace() or content_after_tag == ".":
        return False
    return True

# 过滤掉answer内容无效的条目
valid_indices = [i for i, example in enumerate(dataset2) if 'output' in example and has_valid_content(example['output'])]
dataset2 = dataset2.select(valid_indices)
print(f"Filtered dataset2 from original size to {len(dataset2)} valid examples")


original_columns2 = dataset2.column_names
dataset2 = dataset2.map(
    lambda x: {
        "prompt" : [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": x["instruction"] if 'instruction' in x else x.get('input', '')},
        ],
        "answer": extract_xml_answer(x["output"]),
    },
    remove_columns=original_columns2
)

# 合并两个数据集，并打乱
dataset = concatenate_datasets([dataset, dataset2])
dataset = dataset.shuffle(seed=42)

print(f"Combined dataset size: {len(dataset)}")

dataset2 size 2008
Filtered dataset2 from original size to 1979 valid examples
Combined dataset size: 9452


### 定义奖励函数

In [7]:
import re

# 定义正则表达式，用来判断模型的输出是否符合格式要求
match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [8]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

In [9]:
# 格式匹配函数
def match_format_exactly(completions, **kwargs):
    """格式判断函数，严格判断格式是否匹配

    Args:
        completions (_type_): _description_

    Returns:
        _type_: Number 0 ｜ 3
    """
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [10]:
def match_format_approximately(completions, **kwargs):
    """弱格式判断奖励，即使没有严格对应，也可以根据使用的标签数量来做出相应的奖励

    Args:
        completions (_type_): _description_

    Returns:
        _type_: Number
    """
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # 数一数看到多少个关键词——如果太多，我们会惩罚你！
        # 如果我们看到1，那么加一些积分！如果更多了，那么就应当扣除一些分
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [11]:
def check_answer(prompts, completions, answer, **kwargs):
    """通过比较提取的答案与参考答案来评估模型响应。
    
    该函数从结构化模型输出中提取答案并与参考答案进行比较，根据匹配质量分配分数：
    - 完全匹配：3.0分
    - 去除空格后匹配：1.5分
    - 数值答案在正确值10%范围内：0.5分
    - 数值答案在正确值20%范围内：0.25分
    - 错误答案：-0.5或-1.0分
    
    参数：
        prompts (list)：提供给模型的对话提示列表
        completions (list)：需要评估的模型生成的回答
        answer (list)：用于比较的参考答案
        **kwargs：额外参数
        
    返回：
        list：基于答案正确性的每个回答的得分
    """
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # 如果完全一致，就给出 3 分 
        if guess == true_answer:
            score += 3.0
        # 如果结果正确，但是有空格，就给1.5分
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # 如果答案接近比率，我们也会奖励它！
            # 即，如果答案在某个范围内，奖励它！
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                # 如果直接异常了，就抛出错误
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

In [12]:
# 对于数学问题，先给数字部分抽取出来
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

['0.34']

In [13]:
def check_numbers(prompts, completions, answer, **kwargs):
    """使用正则表达式从模型输出中提取数字答案并进行评分。
    
    该函数从模型响应中提取数字，并与参考答案进行数值比较。
    如果提取的数字与正确答案完全匹配，将获得1.5分，否则为0分。
    
    参数：
        prompts (list)：提供给模型的对话提示列表
        completions (list)：需要评估的模型生成的回答
        answer (list)：用于比较的参考答案数值
        **kwargs：额外参数
        
    返回：
        list：基于数值匹配的评分列表
    """
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

### 训练部分

In [14]:
max_prompt_length = 256

# 使用 GRPO 训练器，并构造训练器
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    beta = 0.0, # 设置为 0 以禁用 KL 散度惩罚 # defaults to 0.04
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # 增加到4，以便更顺滑地训练 #1
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 1000, # 训练步数
    save_steps = 250, # 每50步保存一次
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs_gemma3_1b_it_2", # 输出目录
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


开始训练。期望在训练中，看到reward列的数值增长！

有可能在开始的100步都没有奖励，你可能需要等待150-200步。

In [ ]:
# 创建训练器，并且使用上面给出的 reward function
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,452 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880/1,006,408,832 (0.65% trained)


### 模型测试
#### 默认模型测试

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

NameError: name 'system_prompt' is not defined

#### 保存 Lora

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

In [ ]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

### 保存为完整模型

In [ ]:
# if False: # Change to True to upload finetune
#     model.push_to_hub_merged(
#         "HF_ACCOUNT/gemma-3-finetune", tokenizer,
#         token = "hf_..."
#     )

In [ ]:
# 保存为 GGUF 格式
# if False:
#     model.save_pretrained_gguf(
#         "gemma-3-finetune",
#         quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
#     )

In [ ]:
# if False: # Change to True to upload GGUF
#     model.push_to_hub_gguf(
#         "gemma-3-finetune",
#         quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
#         repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
#         token = "hf_...",
#     )